In [1]:
import pandas as pd
import uuid

In [139]:
def clean_track(track_name:str) -> str:
    clean = track_name.split('(')[0]
    return clean

def split_artist(artists:str) -> list:
    clean = artists.split(', ')
    clean = [i.strip() for i in clean if i.strip()]
    return clean

In [142]:
df = pd.read_csv("data/spotify_data_with_id.csv")
df["clean_track"] = [clean_track(i) for i in df.track_name]
df["artists"] = [split_artist(i) for i in df["artist(s)_name"]]
df = df[df["artists"].str.len() == df["artist_count"]]

df["release_date"] = pd.to_datetime(dict(year=df.released_year, month=df.released_month, day=df.released_day))
df["track_uuid"] = [uuid.uuid4() for i in df.index]

df.drop(columns=["released_year", "released_month", "released_day", "artist_count", "artist(s)_name"], inplace=True)

In [182]:
track_features = ["track_uuid", "track_id", "track_name", "clean_track","release_date", "streams"]
audio_features = ["track_uuid", "bpm", "key", "mode", "danceability_%", "valence_%", "energy_%", "acousticness_%", "instrumentalness_%", "liveness_%", "speechiness_%"]

In [172]:
track = df[track_features]
artist = df[["artists"]].explode("artists").drop_duplicates().rename(columns={"artists":"artist"})
artist["artist_uuid"] = [uuid.uuid4() for i in artist.index]

In [175]:
track_artist = df[["track_uuid","artists"]].explode("artists")\
    .set_index("artists")\
    .join(artist.set_index("artist"))\
    .reset_index()\
    .drop(columns=["artists"])

In [188]:
track.to_csv("data/clean/track.csv", index=False)
df[audio_features].to_csv("data/clean/track_audio.csv", index=False)
artist.to_csv("data/clean/artist.csv", index=False)
track_artist.to_csv("data/clean/edge_track_artist.csv", index=False)

Artist

In [104]:
artist_big = pd.read_csv('../raw/artists.csv').dropna()
artist = pd.read_csv('../clean/artist.csv')

In [147]:
joind = artist.set_index('artist').join(artist_big.set_index("name"), how='left').reset_index()
# artist_big has duplicate names
# for simplicity, let's assume that since we are looking at top hits, that the correct artist is one with the most followers
most_probable = joind[['artist_uuid','followers']].groupby('artist_uuid').max().reset_index().set_index(['artist_uuid','followers'])
joinf = joind.set_index(['artist_uuid','followers']).join(most_probable, how='inner').reset_index()


In [151]:
artist_features = ['artist_uuid','id', 'artist', 'followers', 'popularity']
joinf[artist_features].to_csv('../clean/artist.csv', index=False)

Genre

In [ ]:
joinf

In [161]:
joinf['genres'] = [eval(i) if i==i else None for i in joinf.genres]
genres = joinf.explode('genres')[['genres']].drop_duplicates().dropna()
genres['genre_uuid'] = [uuid.uuid4() for i in genres.index]

In [171]:
artist_genre = joinf[['artist_uuid', 'genres']].explode('genres')\
    .set_index('genres')\
    .join(genres.set_index('genres'))\
    .reset_index()\
    .dropna()\
    .drop(columns=['genres']).drop_duplicates()

artist_genre.to_csv('../clean/edge_artist_genre.csv', index=False)

In [167]:
genres.to_csv('../clean/genre.csv', index=False)